In [2]:
cd C:\Users\venka\OneDrive\Desktop\Project AI

C:\Users\venka\OneDrive\Desktop\Project AI


# Import the Libraries

In [16]:
import pandas as pd
import numpy as np
import random

Import the data

In [60]:
df = pd.read_csv('Indian Cities Database.csv')

In [8]:
df

,City,Lat,Long,country,iso2,State
0,Abohar,30.144533,74.195520,India,IN,Punjab
1,Adilabad,19.400000,78.310000,India,IN,Telangana
2,Agartala,23.836049,91.279386,India,IN,Tripura
3,Agra,27.187935,78.003944,India,IN,Uttar Pradesh
4,Ahmadnagar,19.094571,74.738432,India,IN,Maharashtra
...,...,...,...,...,...,...
208,Vizianagaram,18.113290,83.397743,India,IN,Andhra Pradesh
209,Warangal,17.978423,79.600209,India,IN,Telangana
210,Jorapokhar,23.700000,86.412670,India,IN,Jharkhand
211,Brajrajnagar,21.820000,83.920000,India,IN,Odisha


In [7]:
df.describe()

,Lat,Long
count,213.000000,213.000000
mean,21.734604,79.341943
std,6.394908,5.335423
min,8.177313,69.600868
25%,16.960361,75.853789
50%,22.576882,78.000000
75%,26.884682,81.428497
max,34.318174,94.908370


In [61]:
#sorting by 
df = df.sort_values(by=['City'])

In [62]:
df.reset_index(inplace=True)

In [63]:
df.shape

(213, 7)

In [65]:
df

,index,City,Lat,Long,country,iso2,State
0,0,Abohar,30.144533,74.195520,India,IN,Punjab
1,1,Adilabad,19.400000,78.310000,India,IN,Telangana
2,2,Agartala,23.836049,91.279386,India,IN,Tripura
3,3,Agra,27.187935,78.003944,India,IN,Uttar Pradesh
4,4,Ahmadnagar,19.094571,74.738432,India,IN,Maharashtra
...,...,...,...,...,...,...,...
208,205,Varanasi,25.317740,83.005811,India,IN,Uttar Pradesh
209,206,Vellore,12.905769,79.137104,India,IN,Tamil Nadu
210,207,Vishakhapatnam,17.704052,83.297663,India,IN,Andhra Pradesh
211,208,Vizianagaram,18.113290,83.397743,India,IN,Andhra Pradesh


In [70]:
lat = df['Lat'].tolist()
long = df['Long'].tolist()
cities = df['City'].tolist()

Algorithm to check wether we can reach all the points from any starting point.

In [21]:
def connected(matrix):
    def bfs(city):
        for i,go in enumerate(matrix[city]):
            if i not in visited and go==1:
                visited.add(i)
                bfs(i)
    visited = set()
    bfs(0)
    print(visited)
    if len(visited)==len(matrix):
        return True
    else:
        return False     

In [14]:
matrix = [[0]*213 for _ in range(213)]

In [15]:
for i in range(213):
    for j in range(213):
        if i==j:
            matrix[i][j] = 1

Iterativiely update matrix until we get fully connected graph.

In [18]:
while connected(matrix)==False:
    i = random.randint(0,212)
    j = random.randint(0,212)
    if matrix[i][j] == 0:
        matrix[i][j] = 1

In [26]:
c = 0
for i in range(213):
    for j in range(213):
        if matrix[i][j] == 1:
            c+=1
print('We have total connections of: ',c - 213)

We have total connections of:  1165


# Create distance matrix using OSMR API

In [48]:
distances = [[0]*213 for _ in range(213)]

In [68]:
import requests
import json
# call the OSMR API
for i in range(213):
    for j in range(213):
        if matrix[i][j] == 1:
            try:
                r = requests.get(f"http://router.project-osrm.org/route/v1/car/{long[i]},{lat[i]};{long[j]},{lat[j]}?overview=false""")
                # then you load the response using the json libray
                # by default you get only one alternative so you access 0-th element of the `routes`
                routes = json.loads(r.content)
                route_1 = routes.get("routes")[0]
                distances[i][j] = route_1['distance']/1000
            except:
                distances[i][j] = 5000
    if i%10 ==0:
        print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210


In [73]:
# Prepare ajacency list
adjacency_list = {}
for i,city in enumerate(cities):
    adjacency_list[city] = []
    for j in range(len(cities)):
        if matrix[i][j] == 1:
            adjacency_list[city].append((cities[j],distances[i][j]))

# Make a heuristic matrix. 

We consider heuristic as direct arieal distance between any two cities.

In [33]:
# Python 3 program to calculate Distance Between Two Points on Earth
from math import radians, cos, sin, asin, sqrt
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

# A* Algorithm to find distance between start and goal cities

In [86]:
from collections import deque
 
class Graph:
    def __init__(self, adjac_lis, lat, long, cities):
        self.adjac_lis = adjac_lis
        self.lat = lat
        self.long = long
        self.cities = cities
 
    def get_neighbors(self, v):
        return self.adjac_lis[v]
 
    # This is heuristic function which is having equal values for all nodes
    def h(self, city,stop):
        i = self.cities.index(city)
        j = self.cities.index(stop)
        d=distance(self.lat[i], self.lat[j], self.long[i], self.long[j])
        return d
 
    def a_star_algorithm(self, start, stop):
        # In this open_lst is a list of nodes which have been visited, but who's 
        # neighbours haven't all been always inspected, It starts off with the start 
  #node
        # And closed_lst is a list of nodes which have been visited
        # and who's neighbors have been always inspected
        open_lst = set([start])
        closed_lst = set([])
 
        # poo has present distances from start to all other nodes
        # the default value is +infinity
        poo = {}
        poo[start] = 0
 
        # par contains an adjac mapping of all nodes
        par = {}
        par[start] = start
 
        while len(open_lst) > 0:
            n = None
 
            # it will find a node with the lowest value of f() -
            for v in open_lst:
                if n == None or poo[v] + self.h(v,stop) < poo[n] + self.h(n,stop):
                    n = v;
 
            if n == None:
                print('Path does not exist!')
                return None
 
            # if the current node is the stop
            # then we start again from start
            if n == stop:
                reconst_path = []
 
                while par[n] != n:
                    reconst_path.append(n)
                    n = par[n]
 
                reconst_path.append(start)
 
                reconst_path.reverse()
 
                print('Path found: {}'.format(reconst_path))
                return reconst_path, poo[stop]
 
            # for all the neighbors of the current node do
            for (m, weight) in self.get_neighbors(n):
              # if the current node is not presentin both open_lst and closed_lst
                # add it to open_lst and note n as it's par
                if m not in open_lst and m not in closed_lst:
                    open_lst.add(m)
                    par[m] = n
                    poo[m] = poo[n] + weight
 
                # otherwise, check if it's quicker to first visit n, then m
                # and if it is, update par data and poo data
                # and if the node was in the closed_lst, move it to open_lst
                else:
                    if poo[m] > poo[n] + weight:
                        poo[m] = poo[n] + weight
                        par[m] = n
 
                        if m in closed_lst:
                            closed_lst.remove(m)
                            open_lst.add(m)
 
            # remove n from the open_lst, and add it to closed_lst
            # because all of his neighbors were inspected
            open_lst.remove(n)
            closed_lst.add(n)
 
        print('Path does not exist!')
        return None

In [87]:
graph1 = Graph(adjacency_list,lat,long,cities)
path = graph1.a_star_algorithm('Abohar', 'Warangal')

Path found: ['Abohar', 'Mangalore', 'Warangal']


In [88]:
path

(['Abohar', 'Mangalore', 'Warangal'], 3364.8152)